<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_007_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversational AI - aka Chatbot!

### Install Libraries

In [2]:
# !pip install python-dotenv
# !pip install openai
# !pip install google-generativeai
# !pip install anthropic
# !pip install gradio

### Import Libraries

In [4]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

### Load Environment Variables

In [5]:
# Load the environment variables from the .env file
load_dotenv('/content/API_KEYS.env')  # Ensure this is the correct path to your file

# Get the API keys from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

# Check if the keys are loaded correctly and print a portion of them
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[0:10]}...")  # Only print part of the key
else:
    print("OpenAI API key not loaded correctly.")

if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[0:10]}...")
else:
    print("Anthropic API key not loaded correctly.")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[0:10]}...")
else:
    print("Google API key not loaded correctly.")

OpenAI API Key loaded: sk-proj-e1...
Anthropic API Key loaded: sk-ant-api...
Google API Key loaded: AIzaSyDh3a...


In [6]:
import openai
import anthropic
import google.generativeai

# Connect to OpenAI
openai.api_key = openai_api_key  # Set OpenAI API key

# Connect to Anthropic (Claude)
claude = anthropic.Anthropic(api_key=anthropic_api_key)  # Set Anthropic API key

# Connect to Google Generative AI
google.generativeai.configure(api_key=google_api_key)  # Set Google API key

## Message Structure and History

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

We will write a function `chat(message, history)` where:
**message** is the prompt to use
**history** is a list of pairs of user message with assistant's reply

```
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
```

### Major Concepts You Should Be Learning:

1. **Conversation History in ChatGPT**:
   - **How history is handled**: The `history` parameter contains the conversation history between the user and the assistant. This history is crucial because ChatGPT doesn't maintain state across calls. So, the full conversation context needs to be passed each time.
   - **Rebuilding the conversation**: The `for` loop iterates over the history, adding past user messages (`"role": "user"`) and assistant responses (`"role": "assistant"`) to the `messages` list. This maintains the continuity of the conversation by passing the entire message history to the model in each API call.

2. **System and User Messages**:
   - **System message**: As before, the system message sets the tone for the assistant's behavior. It’s added first in the `messages` list.
   - **User messages**: The current user message (along with the conversation history) is added to the `messages` list. This helps provide context to the model about the ongoing interaction.

3. **Streaming Responses**:
   - **Streamed output**: The `stream=True` parameter enables response streaming. Instead of waiting for the entire response, you get real-time chunks of the model's output.
   - **Yielding the response**: The response is constructed incrementally by appending each chunk of text to `response` and then yielding the partial response. This allows you to update the UI (or any interface) in real time as the model generates the response.

4. **Handling Role-based Messages**:
   - **Assistant and user roles**: The API expects messages to have a role. This function adds `"role": "user"` for user inputs and `"role": "assistant"` for the model's previous outputs. By doing this, the API understands the flow of conversation and can generate an appropriate response based on both the assistant's past replies and the user’s queries.

### Key Differences from Previous Code:
- **Conversation History**: This function introduces conversation history, which is a key feature in creating more dynamic and context-aware interactions with the model. The history is structured in pairs (user message, assistant message) and is passed along to give context to the assistant for every new input.
- **Role Assignment**: In previous examples, we only passed the system and user messages. Here, the function handles both user and assistant roles, ensuring the model can respond in context to previous interactions.

### What You Should Focus On:

1. **Maintaining Conversation State**:
   - You need to understand how the conversation history is passed in and rebuilt for every new request. This is important because ChatGPT doesn't remember conversations from previous API calls unless you include the history explicitly.
   
2. **Streaming with Real-time Updates**:
   - Streaming is used here to yield partial responses from the model as they are generated. This is useful for creating a more responsive and dynamic user experience, especially for longer outputs.

3. **Role-based Messages**:
   - The model needs to know who is saying what. The `"role"` field ensures the model can differentiate between the user’s inputs and the assistant’s responses, which is critical for maintaining the conversational flow.



## Chat Function with Chat History

### Example of `history` and `messages`

Suppose the user and assistant have had the following conversation:
1. **User**: "Hello, can you help me with my homework?"
2. **Assistant**: "Of course! What subject are you working on?"
3. **User**: "Math. I'm stuck on algebra."
4. **Assistant**: "Got it. Let's work on it together."

The `history` would look like this:
```python
history = [
    ("Hello, can you help me with my homework?", "Of course! What subject are you working on?"),
    ("Math. I'm stuck on algebra.", "Got it. Let's work on it together.")
]
```

#### Resulting `messages`:
If the current user message is `"How do I solve quadratic equations?"`, then `messages` will be:
```python
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hello, can you help me with my homework?"},
    {"role": "assistant", "content": "Of course! What subject are you working on?"},
    {"role": "user", "content": "Math. I'm stuck on algebra."},
    {"role": "assistant", "content": "Got it. Let's work on it together."},
    {"role": "user", "content": "How do I solve quadratic equations?"}
]
```

### Why Learning How Chat History Works Is Important

1. **Maintaining Context**:
   - Chat history allows the assistant to maintain context across multiple user interactions.
   - Without this, the model would treat each new user message as a standalone request, losing continuity and providing responses that may not make sense in an ongoing conversation.

2. **Creating Coherent Conversations**:
   - By appending previous user and assistant messages, the assistant can refer back to past messages, making the conversation feel more natural and engaging.
   - It allows for more meaningful responses, as the assistant knows what has already been discussed.

3. **Personalization**:
   - If the assistant can remember and refer to earlier parts of a conversation, it can provide more personalized assistance.
   - For example, remembering that a user was working on algebra allows the assistant to continue helping without re-explaining everything.

4. **Debugging and Improving Conversations**:
   - Understanding how history is built and used can help debug issues where the assistant forgets context or provides irrelevant responses.
   - It can also help improve prompt engineering to ensure the assistant has the best possible understanding of the entire conversation.

In summary, the **chat history** feature is fundamental to building more interactive and engaging AI-driven conversations. Learning how to maintain, construct, and pass this history to the LLM is key to creating chatbots that deliver coherent and context-aware responses, making the interaction more useful for users.

# works

In [35]:
import gradio as gr

MODEL = 'gpt-4o-mini'
system_message = "You are a helpful assistant"

def chat(message, history):
    # Start the message list with the system message
    messages = [{"role": "system", "content": system_message}]

    # Iterate through the history, adding user and assistant messages using OpenAI-style format
    for msg in history:
        # Expecting history in OpenAI-style with role and content
        messages.append(msg)

    # Add the latest user message to the conversation
    messages.append({"role": "user", "content": message})

    # Send the message to the OpenAI API with streaming
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# Specify type='messages' to use OpenAI-style role/content format
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d1f68fbd0a108375e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Gradio Chatbot


In [43]:
import openai
import gradio as gr
import json
import os

MODEL = 'gpt-4o-mini'
system_message = "You are a helpful assistant"
history_file = "chat_history.json"

# Function to save chat history to a JSON file
def save_chat_history(history, file_path):
    with open(file_path, "w") as file:
        json.dump(history, file, indent=4)

# Modified chat function
def chat(message, history):
    # Start the message list with the system message
    messages = [{"role": "system", "content": system_message}]

    # Iterate through the history, adding user and assistant messages using OpenAI-style format
    for msg in history:
        # Expecting history in OpenAI-style with role and content
        messages.append(msg)

    # Add the latest user message to the conversation
    messages.append({"role": "user", "content": message})

    # Send the message to the OpenAI API with streaming
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

    # Add the current user message and assistant response to the history
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})

    # Save updated history to a JSON file
    save_chat_history(history, history_file)

# Specify type='messages' to use OpenAI-style role/content format
gr.ChatInterface(fn=chat, type="messages").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7d9e01f3ef67027116.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Print Chat History

In [44]:
import json
import os

def print_chat_history(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            history = json.load(file)

        print("\nChat History:\n" + "=" * 50)
        turn = 1
        for msg in history:
            if msg["role"] == "user":
                # print(f"Turn {turn}:")
                print(f"User: {msg['content']}")
            elif msg["role"] == "assistant":
                print(f"Assistant: {msg['content']}")
                print("-" * 50)
                turn += 1
    else:
        print("No chat history found.")

print_chat_history("/content/chat_history.json")



Chat History:
User: hello there
Assistant: Hello! How can I assist you today?
--------------------------------------------------
User: what is the world largest mammal?
Assistant: The world’s largest mammal is the blue whale (*Balaenoptera musculus*). Blue whales can reach lengths of up to 100 feet (30 meters) and can weigh as much as 200 tons (approximately 181 metric tonnes) or more. They are found in oceans around the world and are known for their distinct blue-gray coloration and incredible size.
--------------------------------------------------
User: What is the world longest river?
Assistant: The title of the world’s longest river is subject to some debate, primarily between the Nile River and the Amazon River. 

1. **Nile River**: Traditionally, the Nile has been considered the longest river in the world, measuring about 6,650 kilometers (4,130 miles) in length. It flows north through northeastern Africa and drains into the Mediterranean Sea.

2. **Amazon River**: Recent studi

This code provides a good example of how to use OpenAI's chatbot API with Gradio, illustrating how to manage **conversation history** and **message formatting** effectively for an interactive chat experience. Let’s break down the **message structure** and **history management**, focusing on key takeaways for understanding how to work with chatbots.

### Key Takeaways for Learning Chatbots with OpenAI:

1. **Message and History Structure**:
   - **Message Format**:
     - The messages used in the chat follow OpenAI's required format for conversations, where each message is a dictionary containing:
       - `"role"`: This specifies the **role** of the message sender, which can be `"system"`, `"user"`, or `"assistant"`.
       - `"content"`: This is the **content** of the message.
     - For example:
       ```python
       {"role": "user", "content": "Hello, what can you do?"}
       {"role": "assistant", "content": "I can help you with answering questions, giving advice, etc."}
       ```
     - The `"role"` and `"content"` structure is crucial because it helps the model understand **who said what** during the conversation.
  
   - **System Message**:
     - A **system message** is added at the start of the conversation to establish the context, tone, or behavior of the assistant. In this case:
       ```python
       {"role": "system", "content": "You are a helpful assistant"}
       ```
     - The system message is like a guide that sets the assistant's behavior throughout the conversation.

2. **Maintaining Conversation Context**:
   - **Iterating Through the History**:
     - The `history` parameter holds the conversation history as a **list of dictionaries.**
     - The loop `for msg in history:` is used to add each previous message (both user and assistant) to the `messages` list.
     - This maintains the full context of the conversation, which is crucial for OpenAI's model to generate relevant responses.
     - History format:
       ```python
       [
           {"role": "user", "content": "Hello"},
           {"role": "assistant", "content": "Hi there! How can I assist you today?"},
           ...
       ]
       ```
     - Keeping track of the entire conversation allows the model to generate coherent and contextually aware responses. Without context, the assistant might not understand the relevance of the user's questions.

3. **Adding User Messages and Generating Responses**:
   - **Adding the Latest User Message**:
     - After the `messages` list has been populated with all prior conversation history, the latest user message is added:
       ```python
       messages.append({"role": "user", "content": message})
       ```
     - This makes sure that the assistant has the latest user input to respond to.

   - **Generating a Response**:
     - The `messages` list is then sent to the OpenAI API:
       ```python
       stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
       ```
     - The `stream=True` option is used to generate responses incrementally, which can make the interaction feel faster and more conversational.

  
4. **Updating History**:
   - **Appending the Conversation to History**:
     - Once a response has been generated, the user message and assistant response are added to `history`:
       ```python
       history.append({"role": "user", "content": message})
       history.append({"role": "assistant", "content": response})
       ```
     - This allows the conversation to be preserved and used for the next turn, providing context for further interactions.


### Key Takeaways:

1. **Message Format Is Crucial**:
   - Use OpenAI’s format for messages: dictionaries with `"role"` and `"content"` as a **list of dictionaries.**
   - Always include a **system message** to set the assistant's behavior, and maintain user and assistant messages for context.

2. **Maintaining Context with History**:
   - Maintaining a complete conversation history helps the model understand the context and respond appropriately.
   - The `history` parameter is passed by Gradio and is updated after every turn to ensure the conversation continues logically.

3. **Gradio Simplifies Chat UI**:
   - **Gradio** provides an easy way to create an interactive user interface, handling the complexities of UI development.
   - By using `type="messages"`, Gradio maintains the expected format and state of the conversation history, which makes integration with OpenAI seamless.


### Code that does not work

In [37]:
def chat(message, history):
    # Start the message list with the system message
    messages = [{"role": "system", "content": system_message}]

    # Iterate through the history, adding previous user and assistant messages
    if history is None:
        history = []
    for msg in history:
        messages.append(msg)

    # Add the latest user message to the conversation
    messages.append({"role": "user", "content": message})

    # Generate response from the OpenAI API
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    ).choices[0].message.content

    # Add the current user message and assistant response to the history
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})

    # Return the response and the updated history
    return response, history

# Launch the Gradio interface
gr.ChatInterface(fn=chat, type="messages").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc9df88cd20f010f65.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### History is Maintained by Gradio

The key differences between the **working code** and the **non-working code** that are causing the issue are mainly related to how the **history** is initialized and managed, as well as **consistency** in how the `history` list is updated and used across different interactions.

Let me explain these differences step-by-step:

### 1. **History Initialization (`if history is None:`)**
- **Non-working code**:
  ```python
  if history is None:
      history = []
  ```
  - This conditional check attempts to initialize the `history` to an empty list if it is `None`. This can lead to issues if `history` is managed improperly across different function calls.
  - The **Gradio state** should be handled automatically between calls, and trying to manually initialize `history` can create inconsistencies.
  - **Potential Issue**: If `history` is incorrectly reset to an empty list or improperly passed, it might not retain the previous conversation, causing the assistant to lose context.

- **Working code**:
  - In the working version, `history` is assumed to be correctly passed by Gradio each time, and there’s no manual initialization of `history` to an empty list.
  - Gradio manages the state and handles `history` automatically, so manually setting it can lead to unexpected behavior.

### 2. **History Structure Consistency**
- **Non-working code**:
  ```python
  # Add the current user message and assistant response to the history
  history.append({"role": "user", "content": message})
  history.append({"role": "assistant", "content": response})
  ```
  - The **history** in this version is updated manually by appending the new user message and assistant response.
  - If there is an inconsistency in the data structure or if `history` is modified in an unexpected way, this can lead to errors when trying to access or use `history` in subsequent function calls.
  - **Key Problem**: If `history` is not passed correctly or if its format changes between function calls, it can result in the error you’re seeing.

- **Working code**:
  - In the working code, there is no explicit initialization of `history` and no direct check for `None`.
  - The `history` is directly iterated over, and every message is appended to the `messages` list in the correct format, ensuring consistency.

### 3. **Role of Gradio State Management (`type="messages"`)**
- When using `gr.ChatInterface(fn=chat, type="messages")`, Gradio expects `history` to be passed as a list of dictionaries, where each dictionary contains `"role"` and `"content"`.
- In the **working version**, the code directly uses `history` as it is provided by Gradio, assuming it is in the correct format (`list of dictionaries with "role" and "content"`).
- In the **non-working version**, manually manipulating `history` (`if history is None: history = []`) might be leading to inconsistencies in how Gradio expects the data.

### Summary of Key Differences:

1. **Manual Initialization of `history`**:
   - In the non-working version, `history` is manually initialized (`if history is None: history = []`).
   - This can lead to losing previous messages or having an empty history, disrupting the conversation flow.

2. **State Handling by Gradio**:
   - In the working version, Gradio manages the state automatically and keeps `history` in the proper format.
   - By avoiding manual state initialization, the working version ensures the correct state is used across each interaction.

3. **Consistency in Message Format**:
   - **Working Version**: Simply iterates through `history` as-is, adding each entry to the `messages` list, keeping the format consistent throughout.
   - **Non-working Version**: Attempts to initialize `history` and then modify it manually, which can lead to inconsistencies in the format expected by Gradio or OpenAI.

### Key Takeaway:
- Let **Gradio** handle the **conversation state** (`history`). It automatically provides the history to the `chat` function, and this history is correctly formatted (`list of dictionaries`).
- Avoid manually initializing or changing `history` within the `chat` function, as it can lead to discrepancies between how Gradio manages the state and how your function modifies it.
- Always maintain a consistent format for `history` to ensure smooth conversation flow. Gradio will pass `history` in the required format, and as long as you append new entries using the same format (`{"role": "user" or "assistant", "content": "..."}`), it will work properly.

The working code keeps everything straightforward and consistent, whereas the non-working code introduces manual steps that can disrupt Gradio's internal state handling and lead to errors.

The `history` variable is not explicitly instantiated within the function or in the script itself. This can lead to confusion as it seems like `history` appears out of nowhere. Let me explain what's happening and why the `history` variable is functional in the context of this code.

### Where `history` Comes From
- The **`history`** parameter in the `chat` function is provided by **Gradio** when it calls the function.
- **Gradio** keeps track of the conversation's history and passes it to the function as an argument during each user interaction.

### Explanation:
1. **Gradio's Handling of Chat History**:
   - **Gradio** is the one managing the chat history.
   - When you use `gr.ChatInterface(fn=chat, type="messages")`, Gradio provides the `history` to the `chat()` function automatically.
   - The `history` parameter in this context is maintained by Gradio across multiple interactions between the user and the chatbot.

2. **How `history` Is Populated**:
   - In the first interaction, **`history`** will be an empty list because there is no prior conversation.
   - After the first user message, the response is generated by the model and appended to the `history` list.
   - This updated history is passed back to Gradio, which in turn passes it to the function on subsequent interactions.

3. **How Gradio Manages It**:
   - When a new user message is entered, Gradio will call the `chat()` function again.
   - It will pass the accumulated history as an argument, allowing the assistant to continue from where the conversation left off.
   - Essentially, **Gradio** persists the `history` state between different user inputs, which is why it seems like `history` is never instantiated explicitly in the code.

### Example Flow:
1. **First User Message**:
   - The `chat()` function is called by Gradio.
   - `history` is initially an empty list (`[]`).
   - After generating the assistant's response, the pair `(user_message, response)` is appended to `history`.
   - `history` is saved by Gradio for future interactions.

2. **Second User Message**:
   - Gradio calls `chat()` again, but this time it provides the `history` list from the previous interaction.
   - `history` now contains the previous message-response pairs, allowing the assistant to maintain context.

### Key Takeaways for Learning:
- **`history` Is Managed by Gradio**: The `history` variable is not defined manually in your script because it is managed and passed by **Gradio**. It retains the state of the conversation across multiple calls to the `chat` function.


### Example with Different Parameter Names:
You could rename `message` and `history` to something else, and it would still work with Gradio, as long as they are in the correct order.

- You **don't need to use the exact keywords** like `chat(message, history)`.
- What matters is that you pass **two arguments** to your function: one for the **user’s message** and one for the **conversation history**.
- You can rename the function and parameters to anything descriptive, as long as they match the expected inputs for Gradio.

This flexibility helps you make your code more readable and descriptive without being bound to specific parameter names.



## Chat Function with Shopping Assistant

In [45]:
import gradio as gr

MODEL = 'gpt-4o-mini'
system_message = "You are a helpful assistant"

def chat(message, history):
    # Start the message list with the system message
    messages = [{"role": "system", "content": system_message}]

    # Iterate through the history, adding user and assistant messages using OpenAI-style format
    for msg in history:
        # Expecting history in OpenAI-style with role and content
        messages.append(msg)

    # Add the latest user message to the conversation
    messages.append({"role": "user", "content": message})

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    # Send the message to the OpenAI API with streaming
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

# More detailed system message for the assistant behavior
system_message = (
    "You are a helpful assistant in a clothing store. Your goal is to gently encourage "
    "the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. "
    "For example, if the customer says, 'I'm looking to buy a hat,' you could reply with something like, "
    "'Wonderful - we have lots of hats, including several that are part of our sales event.' "
    "Encourage the customer to buy hats if they are unsure what to get."
)

# Specify type='messages' to use OpenAI-style role/content format
gr.ChatInterface(fn=chat, type="messages").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04317974f1695a4d32.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [26]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

### Alternate System Role

In [27]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d652f58bacb83fc3e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Add Context

In [46]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    if 'belt' in message: # add context about belts
        messages.append({"role": "system", "content": "For added context, the store does not sell belts, \
                        but be sure to point out other items on sale"})

    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response


gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://594e9f0b0b747f64fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
